# 14.1 词嵌入（Word2vec）
- **目录**
  - 14.1.1 独热编码向量的缺陷
  - 14.1.2 自监督的word2vec
  - 14.1.3 跳元模型（Skip-Gram）
    - 14.1.3.1 训练
  - 14.1.4 连续词袋（CBOW）模型
    - 14.1.4.1 训练


- 自然语言是用来表达人脑思维的复杂系统。
- 在这个系统中，词是意义的**基本单元**。
- 顾名思义，**词向量**是用于表示单词意义的向量，
并且还可以被认为是单词的特征向量或表示。
- 将单词映射到实向量的技术称为**词嵌入**。
- 近年来，词嵌入逐渐成为自然语言处理的基础知识。

## 14.1.1 独热编码向量的缺陷

- 在 8.5节中，我们使用独热向量来表示词（字符就是单词）。
- 假设词典中不同词的数量（词典大小）为$N$，每个词对应一个从$0$到$N−1$的不同整数（索引）。
- 为了得到索引为$i$的任意词的独热向量表示，我们创建了一个全为0的长度为$N$的向量，并将位置$i$的元素设置为1。这样，每个词都被表示为一个长度为$N$的向量，可以直接由神经网络使用。
- 虽然独热向量很容易构建，但它们通常不是一个好的选择。一个主要原因是**独热向量不能准确表达不同词之间的相似度**，比如我们经常使用的“余弦相似度”。
- 对于向量$\mathbf{x}, \mathbf{y} \in \mathbb{R}^d$，它们的余弦相似度是它们之间角度的余弦：

$$\frac{\mathbf{x}^\top \mathbf{y}}{\|\mathbf{x}\| \|\mathbf{y}\|} \in [-1, 1].\tag{14.1.1}$$

- 由于任意两个不同词的独热向量之间的**余弦相似度为0**，所以独热向量不能编码词之间的相似性。

## 14.1.2 自监督的word2vec

- [word2vec](https://code.google.com/archive/p/word2vec/)工具是为了解决上述问题而提出的。
- 它将每个词映射到一个**固定长度的向量**，这些向量能更好地表达不同词之间的相似性和类比关系。
- word2vec工具包含两个模型，即**跳元模型（skip-gram）** 和**连续词袋（CBOW）** 。
- 对于在语义上有意义的表示，它们的训练依赖于条件概率，**条件概率可以被看作是使用语料库中一些词来预测另一些单词**。
- 由于是不带标签的数据，因此跳元模型和连续词袋都是**自监督模型**。
- 下面，我们将介绍这两种模式及其训练方法。

## 14.1.3 跳元模型（Skip-Gram）

跳元模型假设**一个词可以用来在文本序列中生成其周围的单词**。以文本序列“the”、“man”、“loves”、“his”、“son”为例。假设**中心词**选择“loves”，并将**上下文窗口**设置为2，如图14.1.1所示，给定中心词“loves”，跳元模型考虑生成<b>上下文词</b>“the”、“man”、“his”、“son”的条件概率：

$$P(\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}\mid\textrm{"loves"}). \tag{14.1.2}$$

假设上下文词是在给定中心词的情况下独立生成的（即**条件独立性**）。在这种情况下，上述条件概率可以重写为：

$$P(\textrm{"the"}\mid\textrm{"loves"})\cdot P(\textrm{"man"}\mid\textrm{"loves"})\cdot P(\textrm{"his"}\mid\textrm{"loves"})\cdot P(\textrm{"son"}\mid\textrm{"loves"}). \tag{14.1.3}$$
<center><img src='../img/skip-gram.svg'></center>
<center>图14.1.1 跳元模型考虑了在给定中心词的情况下<b>生成</b>周围上下文词的<b>条件概率</b></center><br>

在跳元模型中，每个词都有两个$d$维向量表示，用于计算条件概率。更具体地说，对于词典中索引为$i$的任何词，分别用$\mathbf{v}_i\in\mathbb{R}^d$和$\mathbf{u}_i\in\mathbb{R}^d$表示其用作<b>中心词</b>和<b>上下文词</b>时的两个向量。给定中心词$w_c$（词典中的索引$c$），生成任何上下文词$w_o$（词典中的索引$o$）的条件概率可以通过对向量点积的softmax操作来建模：

$$P(w_o \mid w_c) = \frac{\text{exp}(\mathbf{u}_o^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}, \tag{14.1.4}$$


其中**词表索引集**$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。假设上下文词是在给定任何中心词的情况下**独立生成**的。对于上下文窗口$m$，跳元模型的似然函数是在**给定任何中心词的情况下生成所有上下文词的概率**：

$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}), \tag{14.1.5}$$

其中可以省略小于$1$或大于$T$的任何时间步，确保模型在训练时不会越界访问无效的词。

### 14.1.3.1 训练

**跳元模型参数是词表中每个词的中心词向量和上下文词向量**。在训练中，我们通过最大化似然函数（即极大似然估计）来学习模型参数。这相当于最小化以下损失函数：

$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)}).  \tag{14.1.6}$$

**当使用随机梯度下降来最小化损失时，在每次迭代中可以随机抽样一个较短的子序列来计算该子序列的（随机）梯度，以更新模型参数**。为了计算该（随机）梯度，我们需要获得对数条件概率关于中心词向量和上下文词向量的梯度。通常，根据公式14.1.4，(对公式14.1.4取对数后)涉及中心词$w_c$和上下文词$w_o$的对数条件概率为：

$$\log P(w_o \mid w_c) =\mathbf{u}_o^\top \mathbf{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)\right).  \tag{14.1.7}$$


通过微分，我们可以获得其相对于中心词向量$\mathbf{v}_c$的梯度为

$$\begin{aligned}\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \mathbf{v}_c}&= \mathbf{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\mathbf{u}_j^\top \mathbf{v}_c)\mathbf{u}_j}{\sum_{i \in \mathcal{V}} \exp(\mathbf{u}_i^\top \mathbf{v}_c)}\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\mathbf{u}_j^\top \mathbf{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \mathbf{v}_c)}\right) \mathbf{u}_j\\&= \mathbf{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \mathbf{u}_j.\end{aligned} \tag{14.1.8}$$


注意，公式14.1.8中的计算需要词典中以$w_c$为中心词的所有词的条件概率。其他词向量的梯度可以以相同的方式获得。

对词典中索引为$i$的词进行训练后，得到$\mathbf{v}_i$（作为中心词）和$\mathbf{u}_i$（作为上下文词）两个词向量。在自然语言处理应用中，跳元模型的中心词向量通常用作词表示。

- **要点：**
  - **模型参数**:
    - 跳元模型的参数是词典中每个词的**中心词向量**和**上下文词向量**。
  - **目标函数**:
    - 通过最大化似然函数来学习模型参数，这等同于最小化损失函数（公式14.1.6）。
  - **优化方法**:
    - 在训练过程中，可以使用**随机梯度下降**来最小化损失。
    - 在每次迭代中，可以随机选择一个较短的子序列来计算其随机梯度，并更新模型参数。
  - **条件概率的对数**:
    - 涉及中心词$w_c$ 和上下文词$w_o$的对数条件概率可以由公式14.1.7定义。
  - **梯度计算**:
    - 根据公式14.1.8，可以计算对数条件概率相对于中心词向量 $\mathbf{v}_c$ 的梯度。
    - 这需要计算以$w_c$ 为中心词的词典中所有词的条件概率。
    - 其他词向量的梯度也可以以同样的方式计算。
  - **词的表示**:
    - 训练后，我们会得到每个词的两个词向量：一个作为中心词的向量，另一个作为上下文词的向量。
    - 在多数自然语言处理应用中，跳元模型的中心词向量常用作词的表示。

## 14.1.4 连续词袋（CBOW）模型

**连续词袋（Continuous Bag-of-Words, CBOW）** 模型类似于跳元模型。与跳元模型的主要区别在于，**连续词袋模型假设中心词是基于其在文本序列中的周围上下文词生成的**。例如，在文本序列“the”、“man”、“loves”、“his”、“son”中，在“loves”为中心词且上下文窗口为2的情况下，连续词袋模型考虑基于上下文词“the”、“man”、“him”、“son”（如图14.1.2所示）生成中心词“loves”的条件概率，即：

$$P(\textrm{"loves"}\mid\textrm{"the"},\textrm{"man"},\textrm{"his"},\textrm{"son"}).  \tag{14.1.9}$$
<center><img src='../img/cbow.svg'></center>
<center>图14.1.2 连续词袋模型考虑了给定周围上下文词生成中心词条件概率</center><br>

由于连续词袋模型中存在多个上下文词，因此在**计算条件概率时对这些上下文词向量进行平均**。具体地说，对于字典中索引$i$的任意词，分别用$\mathbf{v}_i\in\mathbb{R}^d$和$\mathbf{u}_i\in\mathbb{R}^d$表示用作**上下文词**和**中心词**的两个向量（符号与跳元模型中相反）。给定上下文词$w_{o_1}, \ldots, w_{o_{2m}}$（在词表中索引是$o_1, \ldots, o_{2m}$）生成任意中心词$w_c$（在词表中索引是$c$）的条件概率可以由以下公式建模:

$$P(w_c \mid w_{o_1}, \ldots, w_{o_{2m}}) = \frac{\text{exp}\left(\frac{1}{2m}\mathbf{u}_c^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}{ \sum_{i \in \mathcal{V}} \text{exp}\left(\frac{1}{2m}\mathbf{u}_i^\top (\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}}) \right)}. \tag{14.1.10}$$


为了简洁起见，我们设为$\mathcal{W}_o= \{w_{o_1}, \ldots, w_{o_{2m}}\}$和$\bar{\mathbf{v}}_o = \left(\mathbf{v}_{o_1} + \ldots, + \mathbf{v}_{o_{2m}} \right)/(2m)$。那么公式14.1.10可以简化为：

$$P(w_c \mid \mathcal{W}_o) = \frac{\exp\left(\mathbf{u}_c^\top \bar{\mathbf{v}}_o\right)}{\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)}. \tag{14.1.11}$$

给定长度为$T$的文本序列，其中时间步$t$处的词表示为$w^{(t)}$。对于上下文窗口$m$，连续词袋模型的似然函数是在给定其上下文词的情况下生成所有中心词的概率：

$$ \prod_{t=1}^{T}  P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}). \tag{14.1.12}$$

### 14.1.4.1 训练

训练连续词袋模型与训练跳元模型几乎是一样的。连续词袋模型的最大似然估计等价于最小化以下损失函数：

$$  -\sum_{t=1}^T  \text{log}\, P(w^{(t)} \mid  w^{(t-m)}, \ldots, w^{(t-1)}, w^{(t+1)}, \ldots, w^{(t+m)}). \tag{14.1.13}$$

请注意，

$$\log\,P(w_c \mid \mathcal{W}_o) = \mathbf{u}_c^\top \bar{\mathbf{v}}_o - \log\,\left(\sum_{i \in \mathcal{V}} \exp\left(\mathbf{u}_i^\top \bar{\mathbf{v}}_o\right)\right). \tag{14.1.14}$$

通过微分，我们可以获得其关于任意上下文词向量$\mathbf{v}_{o_i}$（$i = 1, \ldots, 2m$）的梯度，如下：

$$\frac{\partial \log\, P(w_c \mid \mathcal{W}_o)}{\partial \mathbf{v}_{o_i}} = \frac{1}{2m} \left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} \frac{\exp(\mathbf{u}_j^\top \bar{\mathbf{v}}_o)\mathbf{u}_j}{ \sum_{i \in \mathcal{V}} \text{exp}(\mathbf{u}_i^\top \bar{\mathbf{v}}_o)} \right) = \frac{1}{2m}\left(\mathbf{u}_c - \sum_{j \in \mathcal{V}} P(w_j \mid \mathcal{W}_o) \mathbf{u}_j \right). \tag{14.1.15}$$


其他词向量的梯度可以以相同的方式获得。与跳元模型不同，**连续词袋模型通常使用上下文词向量作为词表示**。

------------
- **说明：连续词袋模型的训练方法与步骤**
  - **连续词袋模型的训练方法：**
    - （1） **初始化**：
      - 随机初始化所有词向量。
      - CBOW有两组词向量：一组用于上下文，另一组用于中心词。
    - （2）**滑动窗口处理文本数据**：
      - 对于文本中的每一个中心词，找到其前后$m$个上下文词。
      - 例如，对于句子“The man loves his son”，如果选择“loves”作为中心词且$m=2$，则上下文词是“The”、“man”、“his”和“son”。
    - （3）**计算上下文词的平均词向量**：
      - 对所有上下文词向量求平均，得到$\bar{\mathbf{v}}_o$。
    - （4）**计算条件概率**：
      - 使用公式14.1.11计算给定上下文词时中心词的条件概率。
    - （5）**计算损失**：
      - 使用公式14.1.14，计算给定上下文词的情况下生成中心词的负对数概率。
      - 累加这些值以获得整个数据集的总损失。
    - （6）**梯度下降**：
      - 使用公式14.1.15计算损失函数相对于词向量的梯度。
      - 根据计算的梯度更新词向量。
    - （7）**迭代**：
      - 重复上述步骤2-6，直到损失函数的值收敛或达到预定的迭代次数。
    - （8）**获取词表示**：
      - 一旦模型被训练完成，通常使用上下文词向量作为词的表示。
  - **连续词袋模型的训练步骤：**
    - （1）**数据处理**：将文本数据转化为中心词和其对应的上下文词。
    - （2）**初始化词向量**：通常使用较小的随机值。
    - （3）**对于每一个中心词和其对应的上下文词**：
      - 计算上下文词的平均词向量$\bar{\mathbf{v}}_o$。
      - 使用Softmax计算给定上下文词时中心词的条件概率。
      - 计算损失函数的值。
      - 使用梯度下降计算词向量的梯度，并更新词向量。
    - （4）**检查是否满足停止条件**：这可能是损失函数的值小于某个阈值、达到预定的迭代次数或其他某个条件。
    - （5）**返回词向量**：使用上下文词向量作为词的表示。
  - 这些方法和步骤提供了一个连续词袋模型的高级视图。
    - 这个模型通常比跳元模型更快收敛，因为它每次都是预测一个词（中心词）而不是$2m$个词（在大小为$m$的上下文窗口中）。

-----------

## 小结

* 词向量是用于表示单词意义的向量，也可以看作是词的特征向量。将词映射到实向量的技术称为词嵌入。
* word2vec工具包含跳元模型和连续词袋模型。
* 跳元模型假设一个单词可用于在文本序列中，生成其周围的单词；而连续词袋模型假设基于上下文词来生成中心单词。

-----------
- **附录：**
- **（1）Skip-Gram跳元模型实现示例**

In [25]:
import numpy as np
from collections import Counter

# 示例文本数据
text = "the man loves his son and the man loves his daughter and the woman loves her son"

# 超参数设置
EMBEDDING_DIM = 50  # 词向量维度 (d)
CONTEXT_WINDOW = 2   # 上下文窗口大小 (m)
LEARNING_RATE = 0.01
EPOCHS = 100
BATCH_SIZE = 2

# 数据预处理
def preprocess(text):
    # 分词
    words = text.lower().split()
    # 构建词汇表
    word_counts = Counter(words)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    vocab_size = len(vocab)
    
    # 创建word2idx和idx2word映射
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    
    return words, word2idx, idx2word, vocab_size

# 创建训练数据
def create_training_data(words, word2idx, window_size):
    training_data = []
    for i, word in enumerate(words):
        center_word = word2idx[word]
        # 获取上下文窗口内的词
        for j in range(i - window_size, i + window_size + 1):
            if j != i and 0 <= j < len(words):
                context_word = word2idx[words[j]]
                training_data.append((center_word, context_word))
    return training_data

# 预处理数据
words, word2idx, idx2word, vocab_size = preprocess(text)
training_data = create_training_data(words, word2idx, CONTEXT_WINDOW)

print(f"词汇表大小: {vocab_size}")
print(f"训练样本数: {len(training_data)}")
print("词汇表:", word2idx)

# 初始化词向量
# 中心词向量 (v_i) 和上下文词向量 (u_i)
center_embeddings = np.random.uniform(-0.5/EMBEDDING_DIM, 0.5/EMBEDDING_DIM, (vocab_size, EMBEDDING_DIM))
context_embeddings = np.random.uniform(-0.5/EMBEDDING_DIM, 0.5/EMBEDDING_DIM, (vocab_size, EMBEDDING_DIM))

# 计算softmax概率 (公式14.1.4)
def compute_softmax(v_c, context_embeddings):
    # v_c: 中心词向量 (1, embedding_dim)
    # context_embeddings: 所有上下文词向量 (vocab_size, embedding_dim)
    scores = np.exp(np.dot(context_embeddings, v_c))  # (vocab_size,)
    return scores / np.sum(scores)  # 归一化得到概率分布

# 计算梯度和更新参数 (公式14.1.8)
def compute_gradients_and_update(center_word_idx, context_word_idx):
    global center_embeddings, context_embeddings
    
    # 获取当前向量
    v_c = center_embeddings[center_word_idx]  # (embedding_dim,)
    u_o = context_embeddings[context_word_idx]  # (embedding_dim,)
    
    # 计算所有词的条件概率 (公式14.1.4)
    probs = compute_softmax(v_c, context_embeddings)  # (vocab_size,)
    
    # 计算v_c的梯度 (公式14.1.8)
    # 第一项: u_o
    # 第二项: sum_j (P(w_j|w_c) * u_j)
    second_term = np.zeros(EMBEDDING_DIM)
    for j in range(vocab_size):
        second_term += probs[j] * context_embeddings[j]
    
    grad_v_c = u_o - second_term  # (embedding_dim,)
    
    # 更新中心词向量
    center_embeddings[center_word_idx] += LEARNING_RATE * grad_v_c
    
    # 计算u_o的梯度：v_c - P(w_o|w_c) * v_c
    grad_u_o = (1 - probs[context_word_idx]) * v_c

    # 更新当前上下文词向量
    # 对偶学习机制：中心词和上下文词互相调整彼此的向量表示
    context_embeddings[context_word_idx] += LEARNING_RATE * grad_u_o

    # 更新其他词向量 (负样本)
    # 当j！= o时的梯度，即负样本的梯度和u_o的梯度不同
    # 其梯度为： -P(w_j|w_c) * v_c
    for j in range(vocab_size):
        if j != context_word_idx:
            context_embeddings[j] -= LEARNING_RATE * probs[j] * v_c

# 训练函数
def train(training_data, epochs, batch_size):
    losses = []
    
    for epoch in range(epochs):
        total_loss = 0
        # 随机打乱数据
        np.random.shuffle(training_data)
        
        # 分批训练
        for i in range(0, len(training_data), batch_size):
            batch = training_data[i:i+batch_size]
            if not batch:
                continue
            
            batch_loss = 0
            for center_word_idx, context_word_idx in batch:
                # 获取当前向量
                v_c = center_embeddings[center_word_idx]
                u_o = context_embeddings[context_word_idx]
                
                # 计算点积
                dot_product = np.dot(u_o, v_c)
                
                # 计算softmax分母
                exp_scores = np.exp(np.dot(context_embeddings, v_c))
                sum_exp = np.sum(exp_scores)
                
                # 计算对数概率 (公式14.1.7)
                log_prob = dot_product - np.log(sum_exp)
                batch_loss += -log_prob  # 负对数似然
                
                # 计算梯度并更新参数
                compute_gradients_and_update(center_word_idx, context_word_idx)
            
            avg_batch_loss = batch_loss / len(batch)
            total_loss += avg_batch_loss
        
        avg_loss = total_loss / (len(training_data) / batch_size)
        losses.append(avg_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')
    
    return losses

# 训练模型
losses = train(training_data, EPOCHS, BATCH_SIZE)

# 获取训练后的词向量
word_vectors = {idx2word[idx]: center_embeddings[idx] for idx in idx2word}

print("\n训练后的词向量:")
for word, vector in word_vectors.items():
    print(f"{word}: {vector[:5]}...")  # 只显示前5维

# 计算并打印几个词的相似度
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("\n词相似度:")
words_to_compare = ['man', 'woman', 'son', 'daughter', 'loves']
for i in range(len(words_to_compare)):
    for j in range(i+1, len(words_to_compare)):
        word1 = words_to_compare[i]
        word2 = words_to_compare[j]
        if word1 in word_vectors and word2 in word_vectors:
            sim = cosine_similarity(word_vectors[word1], word_vectors[word2])
            print(f"{word1} vs {word2}: {sim:.3f}")

词汇表大小: 9
训练样本数: 62
词汇表: {'the': 0, 'loves': 1, 'man': 2, 'his': 3, 'son': 4, 'and': 5, 'daughter': 6, 'woman': 7, 'her': 8}
Epoch [10/100], Loss: 2.1970
Epoch [20/100], Loss: 2.1966
Epoch [30/100], Loss: 2.1958
Epoch [40/100], Loss: 2.1938
Epoch [50/100], Loss: 2.1894
Epoch [60/100], Loss: 2.1790
Epoch [70/100], Loss: 2.1563
Epoch [80/100], Loss: 2.1130
Epoch [90/100], Loss: 2.0508
Epoch [100/100], Loss: 1.9879

训练后的词向量:
the: [-0.1091 -0.0243 -0.0211 -0.0497 -0.1375]...
loves: [ 0.1540  0.0182 -0.0124  0.0380  0.0528]...
man: [ 0.2137  0.0275 -0.1708  0.0969  0.1265]...
his: [-0.0974 -0.0353 -0.0191 -0.0487 -0.1340]...
son: [ 0.0777  0.0197 -0.0691  0.0433  0.0478]...
and: [ 0.1526  0.0092 -0.0548  0.0630  0.0792]...
daughter: [ 0.1037  0.0178 -0.0870  0.0454  0.0619]...
woman: [ 0.0151  0.0018 -0.0286  0.0177  0.0240]...
her: [-0.0122 -0.0006  0.0264 -0.0079 -0.0534]...

词相似度:
man vs woman: 0.650
man vs son: 0.813
man vs daughter: 0.967
man vs loves: 0.032
woman vs son: 0.947
woman vs

- **（2）CBOW模型实现示例**

In [39]:
import numpy as np
from collections import Counter

# 示例文本数据
text = "the man loves his son and the man loves his daughter and the woman loves her son"

# 超参数设置
EMBEDDING_DIM = 50  # 词向量维度 (d)
CONTEXT_WINDOW = 2   # 上下文窗口大小 (m)
LEARNING_RATE = 0.01
EPOCHS = 100
BATCH_SIZE = 2

# 数据预处理
def preprocess(text):
    # 分词
    words = text.lower().split()
    # 构建词汇表
    word_counts = Counter(words)
    vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    vocab_size = len(vocab)
    
    # 创建word2idx和idx2word映射
    word2idx = {word: idx for idx, word in enumerate(vocab)}
    idx2word = {idx: word for idx, word in enumerate(vocab)}
    
    return words, word2idx, idx2word, vocab_size

# 预处理数据
words, word2idx, idx2word, VOCAB_SIZE = preprocess(text)

# 初始化词向量
# 上下文词向量矩阵 (公式中的v_i)
context_vectors = np.random.randn(VOCAB_SIZE, EMBEDDING_DIM) * 0.01
# 中心词向量矩阵 (公式中的u_i)
center_vectors = np.random.randn(VOCAB_SIZE, EMBEDDING_DIM) * 0.01

# 创建训练数据
def create_cbow_data(words, word2idx, window_size=2):
    data = []
    for i in range(window_size, len(words) - window_size):
        # 获取上下文词索引 (公式中的w_{o_1},...,w_{o_{2m}})
        context = [word2idx[words[i+j]] for j in range(-window_size, window_size+1) if j != 0]
        # 获取中心词索引 (公式中的w_c)
        center = word2idx[words[i]]
        data.append((context, center))
    return data

# 生成训练数据
training_data = create_cbow_data(words, word2idx, CONTEXT_WINDOW)

# Softmax函数 (对应公式14.1.11的分母部分)
def softmax(x):
    exp_x = np.exp(x - np.max(x))  # 防止数值溢出
    return exp_x / exp_x.sum()

# CBOW模型训练
def train_cbow(training_data, epochs, batch_size):
    global context_vectors, center_vectors
    
    for epoch in range(epochs):
        total_loss = 0
        np.random.shuffle(training_data)  # 打乱数据
        
        for batch_start in range(0, len(training_data), batch_size):
            # 获取当前batch数据
            batch = training_data[batch_start:batch_start+batch_size]
            
            # 初始化梯度
            grad_context = np.zeros_like(context_vectors)
            grad_center = np.zeros_like(center_vectors)
            
            batch_loss = 0
            
            for context, center in batch:
                # 计算上下文词向量的平均值 (公式14.1.11中的\bar{v}_o)
                context_indices = context
                context_mean = np.mean(context_vectors[context_indices], axis=0)
                
                # 计算预测得分 (公式14.1.11中的u_i^T \bar{v}_o)
                scores = np.dot(center_vectors, context_mean)
                
                # 计算softmax概率 (公式14.1.11)
                probs = softmax(scores)
                
                # 计算交叉熵损失 (公式14.1.14)
                loss = -scores[center] + np.log(np.sum(np.exp(scores)))
                batch_loss += loss
                
                # 计算梯度 (公式14.1.15)
                # 对中心词向量的梯度
                dscores = probs.copy()
                dscores[center] -= 1  # 真实类的梯度                
                # 更新中心词向量梯度 (公式14.1.15右侧的u_c部分)
                grad_center += np.outer(dscores, context_mean)
                
                # 更新上下文词向量梯度 (公式14.1.15)
                dcontext_mean = np.dot(dscores, center_vectors)
                dcontext_mean /= len(context_indices)  # 1/2m
                
                for idx in context_indices:
                    grad_context[idx] += dcontext_mean
            
            # 更新参数
            context_vectors -= LEARNING_RATE * grad_context / batch_size
            center_vectors -= LEARNING_RATE * grad_center / batch_size
            
            total_loss += batch_loss / batch_size
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}, Loss: {total_loss/len(training_data)}")

# 开始训练
train_cbow(training_data, EPOCHS, BATCH_SIZE)

# 获取词向量 (使用上下文词向量作为最终表示)
word_vectors = context_vectors
print("\n训练后的词向量:")
for word, vector in zip(words,context_vectors):
    print(f"{word}: {vector[:5]}...")  # 只显示前5维
    
# 示例：查找相似词
def find_similar_words(word, word2idx, idx2word, word_vectors, top_n=3):
    if word not in word2idx:
        return []
    
    # 获取查询词的向量
    query_vec = word_vectors[word2idx[word]]
    
    # 计算余弦相似度
    similarities = np.dot(word_vectors, query_vec) / (
        np.linalg.norm(word_vectors, axis=1) * np.linalg.norm(query_vec) + 1e-8)
    
    # 获取最相似的词
    similar_indices = np.argsort(-similarities)[1:top_n+1]  # 排除自己
    similar_words = [(idx2word[idx], similarities[idx]) for idx in similar_indices]
    
    return similar_words

# 测试相似词
test_words = ["man", "loves", "son"]
print('\n相似度测试：')
for word in test_words:
    similar = find_similar_words(word, word2idx, idx2word, word_vectors)
    print(f"与'{word}'相似的词: {similar}")

Epoch 10, Loss: 1.0986338437121004
Epoch 20, Loss: 1.0985901083382736
Epoch 30, Loss: 1.0985465301691346
Epoch 40, Loss: 1.0985025037086686
Epoch 50, Loss: 1.0984568626639688
Epoch 60, Loss: 1.0984101293667872
Epoch 70, Loss: 1.098361426152418
Epoch 80, Loss: 1.0983104731738853
Epoch 90, Loss: 1.0982567638267358
Epoch 100, Loss: 1.0981996290066542

训练后的词向量:
the: [-0.0146 -0.0065 -0.0136  0.0017 -0.0065]...
man: [-0.0130  0.0025 -0.0115  0.0019 -0.0009]...
loves: [ 0.0256  0.0003 -0.0125 -0.0105 -0.0125]...
his: [-0.0017  0.0021 -0.0070  0.0086  0.0122]...
son: [ 0.0053  0.0118  0.0228  0.0054 -0.0041]...
and: [-0.0146  0.0211 -0.0044 -0.0109  0.0059]...
the: [ 0.0074 -0.0065 -0.0032  0.0123 -0.0059]...
man: [ 0.0017  0.0026  0.0023  0.0129 -0.0135]...
loves: [-0.0198 -0.0067 -0.0013 -0.0093  0.0014]...

相似度测试：
与'man'相似的词: [('son', 0.164619928476806), ('loves', 0.10033267393542208), ('his', 0.023210785226677844)]
与'loves'相似的词: [('her', 0.159480461727976), ('woman', 0.15162121000492887),

- **CBOW中心词向量梯度计算**

<img src='../img/14_1_1.png' width=800px>  

----------